In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from litequeue import SQLQueue

import sqlite3

In [ ]:
TEST_1 = "key_test_1"
TEST_2 = "key_test_2"

In [ ]:
for conn_type_kwargs in (
    {"filename_or_conn": sqlite3.connect(":memory:")},
    {"filename_or_conn": ":memory:"},
    {"memory": True},
):
    q = SQLQueue(**conn_type_kwargs)
    assert (
        q.conn.isolation_level is None
    ), f"Isolation level not set properly for connection '{conn_type_kwargs}'"

In [ ]:
q = SQLQueue(sqlite3.connect(":memory:"))

q.put("hello")
q.put("world")
q.put("foo")
q.put("bar")

4

In [ ]:
q.pop()

{'message': 'hello', 'task_id': '8ba8d9c02f918726f2393bbdb89aeb05'}

In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7ffbc6df3650>, items=[{'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'hello',
  'status': 1,
  'task_id': '8ba8d9c02f918726f2393bbdb89aeb05'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': None,
  'message': 'world',
  'status': 0,
  'task_id': '21d50ee2588c8e4f8632270809e23da8'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': None,
  'message': 'foo',
  'status': 0,
  'task_id': 'd55fcda6f23d9ba7cc1b76318b621cc3'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': None,
  'message': 'bar',
  'status': 0,
  'task_id': '693a0dd0f06dbb9d1a191d418a946063'}])


In [ ]:
# pop remaining
for _ in range(3):
    q.pop()


assert q.pop() is None

In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7ffbc6df3650>, items=[{'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'hello',
  'status': 1,
  'task_id': '8ba8d9c02f918726f2393bbdb89aeb05'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'world',
  'status': 1,
  'task_id': '21d50ee2588c8e4f8632270809e23da8'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'foo',
  'status': 1,
  'task_id': 'd55fcda6f23d9ba7cc1b76318b621cc3'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'bar',
  'status': 1,
  'task_id': '693a0dd0f06dbb9d1a191d418a946063'}])


In [ ]:
q.put("hello")
q.put("world")
q.put("foo")
q.put("bar")

8

In [ ]:
task = q.pop()

assert task["message"] == "hello"

In [ ]:
q.peek()

{'message': 'world',
 'task_id': '9f11760822527ef422e60abf86b8560d',
 'status': 0,
 'in_time': 1615799709,
 'lock_time': None,
 'done_time': None}

In [ ]:
# next one that is free
assert q.peek()["message"] == "world"

# status = 0 = free
assert q.peek()["status"] == 0

In [ ]:
task["message"], task["task_id"]

('hello', '9780b9a1c3827a5291630e59cf09e3cc')

In [ ]:
q.done(task["task_id"])

8

In [ ]:
q.get(task["task_id"])

{'message': 'hello',
 'task_id': '9780b9a1c3827a5291630e59cf09e3cc',
 'status': 2,
 'in_time': 1615799709,
 'lock_time': 1615799709,
 'done_time': 1615799710}

In [ ]:
already_done = q.get(task["task_id"])

# stauts = 2 = done
assert already_done["status"] == 2

in_time = already_done["in_time"]
lock_time = already_done["lock_time"]
done_time = already_done["done_time"]

assert done_time >= lock_time >= in_time
print(
    f"Task {already_done['task_id']} took {done_time - lock_time} seconds to get done and was in the queue for {done_time - in_time} seconds"
)

Task 9780b9a1c3827a5291630e59cf09e3cc took 1 seconds to get done and was in the queue for 1 seconds


In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7ffbc6df3650>, items=[{'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'hello',
  'status': 1,
  'task_id': '8ba8d9c02f918726f2393bbdb89aeb05'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'world',
  'status': 1,
  'task_id': '21d50ee2588c8e4f8632270809e23da8'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'foo',
  'status': 1,
  'task_id': 'd55fcda6f23d9ba7cc1b76318b621cc3'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'bar',
  'status': 1,
  'task_id': '693a0dd0f06dbb9d1a191d418a946063'},
 {'done_time': 1615799710,
  'in_time': 1615799709,
  'lock_time': 1615799709,
  'message': 'hello',
  'status': 2,
  'task_id': '9780b9a1c3827a5291630e59cf09e3cc'},
 {'done_time': None,
  'in_time': 1615799709,
  'lock_time': None,
  'message': 'world',
  'status': 0,
  'task_id': '9f11760

In [ ]:
assert q.qsize() == 7

In [ ]:
next_one_msg = q.peek()["message"]
next_one_id = q.peek()["task_id"]

task = q.pop()

assert task["message"] == next_one_msg
assert task["task_id"] == next_one_id

In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7ffbc6df3650>, items=[{'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'hello',
  'status': 1,
  'task_id': '8ba8d9c02f918726f2393bbdb89aeb05'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'world',
  'status': 1,
  'task_id': '21d50ee2588c8e4f8632270809e23da8'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'foo',
  'status': 1,
  'task_id': 'd55fcda6f23d9ba7cc1b76318b621cc3'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'bar',
  'status': 1,
  'task_id': '693a0dd0f06dbb9d1a191d418a946063'},
 {'done_time': 1615799710,
  'in_time': 1615799709,
  'lock_time': 1615799709,
  'message': 'hello',
  'status': 2,
  'task_id': '9780b9a1c3827a5291630e59cf09e3cc'},
 {'done_time': None,
  'in_time': 1615799709,
  'lock_time': 1615799711,
  'message': 'world',
  'status': 1,
  'task_id': '9

In [ ]:
q.prune()

In [ ]:
print(q)

SQLQueue(Connection=<sqlite3.Connection object at 0x7ffbc6df3650>, items=[{'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'hello',
  'status': 1,
  'task_id': '8ba8d9c02f918726f2393bbdb89aeb05'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'world',
  'status': 1,
  'task_id': '21d50ee2588c8e4f8632270809e23da8'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'foo',
  'status': 1,
  'task_id': 'd55fcda6f23d9ba7cc1b76318b621cc3'},
 {'done_time': None,
  'in_time': 1615799707,
  'lock_time': 1615799708,
  'message': 'bar',
  'status': 1,
  'task_id': '693a0dd0f06dbb9d1a191d418a946063'},
 {'done_time': None,
  'in_time': 1615799709,
  'lock_time': 1615799711,
  'message': 'world',
  'status': 1,
  'task_id': '9f11760822527ef422e60abf86b8560d'},
 {'done_time': None,
  'in_time': 1615799709,
  'lock_time': None,
  'message': 'foo',
  'status': 0,
  'task_id': 'cd0e9117f85065a

In [ ]:
from string import ascii_lowercase, printable
from random import choice


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [ ]:
q = SQLQueue(":memory:", maxsize=50)

In [ ]:
for i in range(50):

    q.put(random_string(20))

In [ ]:
assert q.qsize() == 50

Make sure an error is raised when the queue has reached its size limit

In [ ]:
import sqlite3

try:
    q.put(random_string(20))
except sqlite3.IntegrityError: # max len reached
    print("test pass")

test pass


In [ ]:
q.pop()

{'message': 'wercpomgaomhylcldvxo',
 'task_id': '6255e21c0b0365a6640374a89a1a1ec3'}

In [ ]:
q.put("hello")

51

In [ ]:
q.empty()

False

In [ ]:
assert q.empty() == False

q2 = SQLQueue(":memory:")

assert q2.empty() == True

**Random benchmarks**

Inserting 100 items in the queue. Since it will run many times, the queue will end up having a lot more than 100 items

In [ ]:
import gc

In-memory SQL queue

In [ ]:
q = SQLQueue(":memory:", maxsize=None)

In [ ]:
gc.collect()

44

In [ ]:
%%timeit -n10000 -r7

q.put(random_string(20))

44.5 µs ± 5.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
q.qsize()

70000

Standard python queue.

In [ ]:
from queue import Queue

In [ ]:
q = Queue()

In [ ]:
gc.collect()

113

In [ ]:
%%timeit -n10000 -r7

q.put(random_string(20))

26.1 µs ± 5.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Persistent SQL queue

In [ ]:
q = SQLQueue("test.queue", maxsize=None)

In [ ]:
gc.collect()

69

In [ ]:
%%timeit -n10000 -r7

q.put(random_string(20))

211 µs ± 32.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
!du -sh test.queue*

9.1M	test.queue
32K	test.queue-shm
5.0M	test.queue-wal


In [ ]:
!rm test.queue*

In [ ]:
assert q.conn.isolation_level is None

Creating and removing tasks

In [ ]:
q = Queue()

In [ ]:
gc.collect()

135

In [ ]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

q.get()

q.task_done()

37.9 µs ± 7.59 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
q = SQLQueue(":memory:", maxsize=None)

In [ ]:
gc.collect()

69

In [ ]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

task = q.pop()

q.done(task["task_id"])

123 µs ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
